In [1]:
import collections

In [2]:
import matplotlib

In [3]:
matplotlib.use('Agg')

In [4]:
%matplotlib qt

In [5]:
import matplotlib.pyplot as plt

In [6]:
import numpy as np

In [7]:
import scipy.stats

In [8]:
import SimpleITK as sitk

In [9]:
from os.path import expanduser, join

In [10]:
from scipy.spatial.distance import euclidean

In [11]:
import imgscroll

---

In [12]:
def show_imgs(imgstack, imgstack2=None):
    """Uses imgscroll to show stack of images with mouse scrolling"""
    X = sitk.GetArrayFromImage(imgstack)
    fig = plt.figure()
    ax = fig.add_subplot(111)
    if imgstack2 is not None:
        X2 = sitk.GetArrayFromImage(imgstack2)
        maskVal = scipy.stats.mode(X2.flatten())[0][0]
        Xmask = np.ma.masked_where(X2 == maskVal, X2)
        tracker = imgscroll.IndexTracker2(ax, X, Xmask)
    else:
        tracker = imgscroll.IndexTracker(ax, X)
    fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
    plt.show()

In [13]:
def input_level_set_3d(featImg, seed2radius):
    """Construct input level set image from user-supplied seed points"""
    setupImg = sitk.Image(featImg.GetSize()[0], featImg.GetSize()[1], sitk.sitkUInt8)

    X2D = sitk.GetArrayFromImage(setupImg)
    for s in seed2radius.keys():
        rowIni, rowEnd = s[0] - seed2radius[s], s[0] + seed2radius[s]
        colIni, colEnd = s[1] - seed2radius[s], s[1] + seed2radius[s]
        for i in range(rowIni, rowEnd+1):
            for j in range(colIni, colEnd+1):
                if euclidean((i,j), s) <= seed2radius[s]:
                    X2D[i,j] = 1
    X = X2D.reshape(1, *X2D.shape).repeat(featImg.GetSize()[2], axis=0)
    
    img = sitk.Cast(sitk.GetImageFromArray(X), featImg.GetPixelIDValue()) * -1 + 0.5
    img.SetSpacing(featImg.GetSpacing())
    img.SetOrigin(featImg.GetOrigin())
    img.SetDirection(featImg.GetDirection())
    return img

In [14]:
def input_level_set_click(featImg, slice2coords):
    RADIUS = 10
    numCols = featImg.GetSize()[0]
    numRows = featImg.GetSize()[1]
    depthSize = featImg.GetSize()[2]
    X = np.zeros((depthSize, numRows, numCols), dtype=np.int)
    
    for n in slice2coords:        
        for c in slice2coords[n]:
            rowIni, rowEnd = c[1] - RADIUS, c[1] + RADIUS
            colIni, colEnd = c[0] - RADIUS, c[0] + RADIUS
            for i in range(rowIni, rowEnd+1):
                for j in range(colIni, colEnd+1):
                    if euclidean((i,j), (c[1], c[0])) <= RADIUS:
                        X[n,i,j] = 1
    
    img = sitk.Cast(sitk.GetImageFromArray(X), featImg.GetPixelIDValue()) * -1 + 0.5
    img.SetSpacing(featImg.GetSpacing())
    img.SetOrigin(featImg.GetOrigin())
    img.SetDirection(featImg.GetDirection())
    return img

In [15]:
class IndexMouseCapture(object):
    def __init__(self, ax, X):
        self.ax = ax
        self.X = X
        self.slices, numrows, numcols = X.shape
        self.ind = 0
        self.ind2coord = collections.defaultdict(list)
        
        self.im = ax.imshow(self.X[self.ind, :, :], cmap=plt.cm.Greys_r)
        self.update()

    def onscroll(self, event):
        if event.button == 'up':
            self.ind = np.clip(self.ind+1, 0, self.slices-1)
        else:
            self.ind = np.clip(self.ind-1, 0, self.slices-1)

        self.update()

    def onclick(self, event):
        ix, iy = int(round(event.xdata)), int(round(event.ydata))
        self.ind2coord[self.ind].append((ix, iy))
        self.update()

    def update(self):
        self.im.set_data(self.X[self.ind, :, :])
        self.ax.set_title('slice %s' %self.ind)
        self.im.axes.figure.canvas.draw()

---

# Read in DICOM images

In [16]:
dicomPath = join(expanduser('~'), 'Documents', 'SlicerDICOMDatabase', 'TCIALocal', '0', 'images', '')
reader = sitk.ImageSeriesReader()
seriesIDread = reader.GetGDCMSeriesIDs(dicomPath)[1]
dicomFilenames = reader.GetGDCMSeriesFileNames(dicomPath, seriesIDread)
reader.SetFileNames(dicomFilenames)
imgSeries = reader.Execute()

In [17]:
# choose a series of slices
imgSlices = imgSeries[:,:,41:50]

In [22]:
show_imgs(imgSlices)

# Filtering

## Curvature anisotropic diffusion

In [18]:
timeStep_, conduct, numIter = (0.06, 9.0, 5)
imgRecast = sitk.Cast(imgSlices, sitk.sitkFloat32)
curvDiff = sitk.CurvatureAnisotropicDiffusionImageFilter()
curvDiff.SetTimeStep(timeStep_)
curvDiff.SetConductanceParameter(conduct)
curvDiff.SetNumberOfIterations(numIter)
imgFilter = curvDiff.Execute(imgRecast)

In [24]:
show_imgs(imgFilter)

# Edge potential

## Gradient magnitude recursive Gaussian

In [19]:
sigma_ = 1.0
imgGauss = sitk.GradientMagnitudeRecursiveGaussian(image1=imgFilter, sigma=sigma_)

In [25]:
show_imgs(imgGauss)

# Feature Image

## Sigmoid mapping

In [26]:
#K1, K2 = 20.0, 8.0
K1, K2 = 10.0, 2.0

In [27]:
alpha_ = (K2 - K1)/6
beta_ = (K1 + K2)/2

sigFilt = sitk.SigmoidImageFilter()
sigFilt.SetAlpha(alpha_)
sigFilt.SetBeta(beta_)
sigFilt.SetOutputMaximum(1.0)
sigFilt.SetOutputMinimum(0.0)
imgSigmoid = sigFilt.Execute(imgGauss)

In [28]:
show_imgs(imgSigmoid)

# Input Level Set

Manually type in the seed coordinates and radii:

In [23]:
#coords = [(118, 286), (135, 254), (202, 75), (169, 89), (145, 209), (142, 147), (252, 58), (205, 119)]
coords = [(118, 286), (135, 254), (145, 209), (142, 147)]

In [24]:
radii = [10, 10, 10, 10]

In [25]:
seed2radius = {tuple(reversed(p[0])): p[1] for p in zip(coords, radii)}

In [26]:
initImg = input_level_set_3d(imgSigmoid, seed2radius)

Alternatively, use the class *IndexMouseCapture* (defined above) to capture coordinates from mouse clicks for seeds. The radii are all assumed to be of the same size at the moment. 

To add circle to image:

    X = sitk.GetArrayFromImage(imgSlice)
    circ = plt.Circle((125, 173), 10, color='b')
    plt.imshow(X, cmap=plt.cm.Greys_r)
    plt.gcf().gca().add_artist(circ)
    plt.show()

To add circle to image interactively and save coordinate:

    def onclick(event):
        ix, iy = int(round(event.xdata)), int(round(event.ydata))
        coord.append((ix, iy))
        circ = plt.Circle((ix, iy), 10, color='b')
        plt.gcf().gca().add_artist(circ)
        im.axes.figure.canvas.draw()
        
    coord = list()
    
    imgSeries = liversegmentation.read_dicom()
    imgSlice = imgSeries[:,:,40]
    
    X = sitk.GetArrayFromImage(imgSlice)
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    fig.canvas.mpl_connect('button_press_event', onclick)
    im = plt.imshow(X, cmap=plt.cm.Greys_r)
    plt.show()

In [34]:
X = sitk.GetArrayFromImage(imgSigmoid)

fig = plt.figure()
ax = fig.add_subplot(111)
tracker = IndexMouseCapture(ax, X)
fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
fig.canvas.mpl_connect('button_press_event', tracker.onclick)

7

In [35]:
initImg = input_level_set_click(imgSigmoid, tracker.ind2coord)

Display the initial input level set:

In [36]:
show_imgs(initImg)

In [37]:
show_imgs(imgSigmoid, initImg)

# Segmentation

## Geodesic Active Contour

In [38]:
gac = sitk.GeodesicActiveContourLevelSetImageFilter()
gac.SetPropagationScaling(1.0)
gac.SetCurvatureScaling(0.2)
gac.SetAdvectionScaling(4.5)
gac.SetMaximumRMSError(0.01)
gac.SetNumberOfIterations(200)

<SimpleITK.SimpleITK.GeodesicActiveContourLevelSetImageFilter; proxy of <Swig Object of type 'itk::simple::GeodesicActiveContourLevelSetImageFilter::Self *' at 0x123460720> >

In [39]:
gacSlices = [gac.Execute(initImg[:,:,z], imgSigmoid[:,:,z]) for z in range(imgSigmoid.GetDepth())]
gac3D = sitk.JoinSeries(gacSlices)

In [31]:
show_imgs(gac3D)

In [40]:
show_imgs(imgSigmoid, gac3D)